In [5]:
import torch
import torch.nn as nn
import numpy as np
import pickle
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from crystallm import (
    GPT,
    GPTConfig,
    CIFTokenizer
)

/home/uccacbo/miniconda3/envs/crystallm_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# check current directory
import os
os.getcwd()

# change directory to where the data is
os.chdir('/home/uccacbo/CrystaLLM')
os.getcwd()

'/home/uccacbo/CrystaLLM'

In [20]:
from crystallm import CIFTokenizer  # Assuming CIFTokenizer is defined in crystallm

# Load the tokenizer (assuming a similar API)
tokenizer = CIFTokenizer()

# Assuming the tokens are stored in a dictionary-like attribute
if hasattr(tokenizer, 'token_to_id'):
    num_tokens = len(tokenizer.token_to_id)
    print(f"Number of tokens in the tokenizer: {num_tokens}")




Number of tokens in the tokenizer: 372


In [4]:
# untar crystallm_v1_large.tar.gz
!tar -xzvf crystallm_v1_large.tar.gz

crystallm_v1_large/
crystallm_v1_large/ckpt.pt


In [17]:
!python bin/train.py --config=config/finetune_LoRA_BG.yaml

Using configuration:
out_dir: pretrained_models/large_model_untouched
ckpt_out_dir: finetuned_models/BG_LoRA_test
eval_interval: 2
log_interval: 2
eval_iters_train: 2
eval_iters_val: 2
eval_only: false
always_save_checkpoint: false
init_from: resume
wandb_log: true
wandb_project: crystallm_CIF_BG_tests
wandb_run_name: BG_large_LoRA
dataset: CIF_BG_proj/BG_large_tokens
gradient_accumulation_steps: 4
batch_size: 4
block_size: 2048
n_layer: 12
n_head: 12
n_embd: 1024
dropout: 0.1
bias: false
learning_rate: 0.0001
max_iters: 4
weight_decay: 0.1
beta1: 0.9
beta2: 0.99
grad_clip: 1.0
decay_lr: true
warmup_iters: 100
lr_decay_iters: 50
min_lr: 1.0e-05
device: cuda
dtype: bfloat16
compile: true
underrep_p: 0.0
validate: true
codecarbon: false
tracker_project: crystallm
metrics_dir: comp_metrics
CarbonTracker: false
LoRA_rank: 4
LoRA_alpha: 4
finetune_method: LoRA
sanity_check: true

Resuming from pretrained_models/large_model_untouched...
Reading start indices from CIF_BG_proj/BG_large_tokens/

In [50]:
!python bin/make_prompt_file.py Na2Cl2 prompts/na2cl2.txt --spacegroup P4/nmm

writing prompt to prompts/na2cl2.txt ...


In [15]:
!python bin/sample.py --config sampling/prompt_config/BG_gen_ft_LoRA.yaml

Using configuration:
out_dir: finetuned_models/BG_LoRA
start: FILE:sampling/prompts/Ti4Ga2Cu2.txt
num_samples: 1
max_new_tokens: 100
temperature: 0.8
top_k: 10
seed: 1337
device: cuda
dtype: bfloat16
compile: false
target: console
generated_dir: sampling/test_generated_cifs_BG
token_resize: true
dataset: CIF_BG_proj/BG_large_tokens

Found dataset vocab_size = 372 (inside CIF_BG_proj/BG_large_tokens/meta.pkl)
Model configuration:
n_layer: 16
n_head: 16
n_embd: 1024
block_size: 2048
bias: false
vocab_size: 372
dropout: 0.1
finetune_method: LoRA

number of parameters: 201.74M
Loaded state_dict with LoRA compatibility.
Model loaded successfully.
Best validation loss (from checkpoint): 0.2797
Training loss not found in checkpoint.
Generating samples...
data_Ti4Ga2Cu2
loop_
_atom_type_symbol
_atom_type_electronegativity
_atom_type_radius
_atom_type_ionic_radius
Ti 1.5400 1.4000 0.8517
Ga 1.8100 1.3000 0.7600
Cu 1.9000 1.3500 0.8200
_symmetry_space_group_name_H-M Immm
_cell_length_a 10.3022
_

In [45]:
!python bin/postprocess.py test_generated_cifs_BG my_processed_cifs_BG

error post-processing CIF file 'sample_1.cif': could not extract space group from:
data_Co4B2Os2
 loop_ Bandgap_eV: _atom_type_symbol _atom_type_electronegativity _atom_type_radius Co,1z8800,1z3500,0z7683 B,2z0400,0z4500,0z4100 Os,2z2000,1z3000,0z6730 _symmetry_space_group_name_H-M,I4/mcm _cell_length_a,3z6036 _cell_length_b,3z6036 _cell_length_c,6z4358 _cell_angle_alpha,90z0000 _cell_angle_beta,90z0000 _cell_angle_gamma,90z0000 _symmetry_Int_Tables_number,119 _chemical_formula_structural,Co2BOs _chemical_formula_sum,/Co4,B2,Os2/ _cell_volume,83z4773 _cell_formula_units_Z,2 loop_ _symmetry_equiv_pos_site_id _symmetry_equiv_pos_as_xyz 1,/_atom_type_oxidation_number',x',y/ loop_ _atom_site_type_symbol _atom_site_label _atom_site_symmetry_multiplicity _atom_site_fract_x _atom_site_fract_y _atom_site_fract_z _atom_site_occupancy Co,Co0,2,0z0000,0z0000,0z0000,1 Co,Co1,2,0z0000,0z5000,0z7500,1 B,B2,2,0z0000,0z5000,0z2500,1 Os,Os3,2,0z0000,0z0000,0z5000,1  data_Ca4Y4Mg4Al4 loop_ Bandgap_eV: _

In [ ]:
# change default directory of jupyter notebook so every time I start it it's in the right place


### Next steps
1. could be that there is no end to the genertation because somehow in the sample.py file the condition to terminate is not found
    
    a. even the max tokens isnt used

2. Maybe when adding the dimnension I didnt add it correctly so all the dimensions got mumble jumbled so the weights and parameter weights arent assigned to the right thing yet (LoRA could fix this? - but for now could try the token resize thing)

3. Maybe need to retrtain all params from scratch

4. Try to assign the start indices

### Lora Implementation

In [8]:
!pip install loralib

In [33]:
import loralib
import os
import torch
from crystallm import GPT, GPTConfig

out_dir = 'pretrained_models/large_model_untouched'
model_args = dict(n_layer=12, n_head=12, n_embd=768, block_size=1024,
                      bias=False, vocab_size=None, dropout=0.1)

ckpt_path = os.path.join(out_dir, "ckpt.pt")
checkpoint = torch.load(ckpt_path, map_location='cuda')
checkpoint_model_args = checkpoint["model_args"]

# Update model arguments based on checkpoint
for k in ["n_layer", "n_head", "n_embd", "block_size", "bias", "vocab_size"]:
    model_args[k] = checkpoint_model_args[k]

gptconf = GPTConfig(**model_args)
model = GPT(gptconf)

number of parameters: 201.74M


In [34]:
print(model)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(371, 1024)
    (wpe): Embedding(2048, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-15): 16 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=1024, out_features=3072, bias=False)
          (c_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=1024, out_features=4096, bias=False)
          (c_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=1024, out_features=371, bias=False)
)


In [41]:
import math
class LoRALayer(torch.nn.Module):
    def __init__(self, in_dim, out_dim, rank, alpha):
        super().__init__()
        std_dev = 1 / torch.sqrt(torch.tensor(rank).float())
        self.A = torch.nn.Parameter(torch.randn(in_dim, rank) * std_dev)
        self.B = torch.nn.Parameter(torch.zeros(rank, out_dim))
        self.alpha = alpha

    def forward(self, x):
        x = self.alpha * (x @ self.A @ self.B)
        return x

In [42]:
class LinearWithLoRA(torch.nn.Module):
    def __init__(self, linear, rank, alpha):
        super().__init__()
        self.linear = linear
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        )

    def forward(self, x):
        return self.linear(x) + self.lora(x)

In [47]:
from functools import partial

# default hyperparameter choices
lora_r = 8
lora_alpha = 16
lora_dropout = 0.05
lora_attn = True
lora_proj = True
lora_head = True


layers = []

assign_lora = partial(LinearWithLoRA, rank=lora_r, alpha=lora_alpha)

for h in model.transformer.h:
    h.attn.c_attn = assign_lora(h.attn.c_attn)
    h.attn.c_proj = assign_lora(h.attn.c_proj)
    h.mlp.c_fc = assign_lora(h.mlp.c_fc)
    h.mlp.c_proj = assign_lora(h.mlp.c_proj)
if lora_head:
    model.transformer.lm_head = assign_lora(model.GPT.lm_head)



AttributeError: 'LinearWithLoRA' object has no attribute 'in_features'

### RSync files to here


In [1]:
!rsync -avz --progress \
uccacbo@myriad.rc.ucl.ac.uk:/home/uccacbo/CrystaLLM/BG_CIF_dataset.tar.gz /home/uccacbo/CrystaLLM/BG_datasets/BG_CIF_dataset.tar.gz

The authenticity of host 'myriad.rc.ucl.ac.uk (193.60.252.107)' can't be established.
ED25519 key fingerprint is SHA256:waqBYWAb+g1lwUGWz8ku4M48McIBWGCdpMoU8l8j6tU.
This key is not known by any other names.
Are you sure you want to continue connecting (yes/no/[fingerprint])? ^C


In [9]:
!pip install psutil pynvml
